# 6.8 Stuff LCEL Chain

https://python.langchain.com/docs/concepts/runnables/

위 페이지를 가서 문서를 살펴보면 chain의 구성요소로 사용되는 것들을 전부 볼 수 있다. 인풋이 뭐가 들어가고, 아웃풋으로는 무엇이 나오는지 등등 말이다. 우리는 이미 Prompt, ChatModel, OutputParser에 대해 알고있다. Retriever는 아직 모르는데, 이제 곧 알게 될 것이다.

위 페이지에서 보면 알 수 있지만, Retriever는 한개의 string을 입력받는다. 질문이나 그와 관련성이 있는 document를 얻기위한 query(질문)를 입력받는것이다.<br>
그리고 Retriever의 출력값은 document들의 List이다.

이제 입출력 타입을 알았으니, 구현(implement)은 아주 쉽다. 해보자.

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0.1)

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("../files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cache_dir = LocalFileStore("./.cache")

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
     embeddings,
     cache_dir,
)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

In [9]:
retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"),
    ("human", "{question}"),
])

chain = retriever | prompt | llm

chain.invoke("What is the name of Oldwyn’s mechanical doll?")

TypeError: list indices must be integers or slices, not str

먼저 구성 요소들의 순서를 생각해보자.

우선, 모든 document를 가져와야 한다. 우리의 query와 관련있는건 전부 가져올 것이다. 그렇다면 Retriever를 첫번째로 넣어주면 된다는 것을 알 수 있다.

retriever는 방금 본 문서에 따르면 document들의 list를 반환할 것이다. 그 document들은 template에 입력되어야 한다. 그래서 두번째 chain으로는 template가 들어가게 된다.

그 다음 구성요소는 prompt와 그 내부에 입력될 데이터를 전달받아야 할 것이다. 즉, LLM을 말한다. 

근데 이것으로는 부족하다. 왜냐하면 우리가 아직 구체적인 동작을 구현하지 않았기 때문이다. 우리가 원하는것은
1. invoke안의 string이 retriever에게 전달이 되고.
2. retriever는 document들의 list를 반환하고.
3. 그 document들의 각 요소는 prompt template의 context로, invoke내의 질문은 question으로 입력되고.

이러한 일련의 동작을 원한다. 우리는 이러한 동작을 실제로 구현하지 않았기 때문에 작동하지 않는것이다. 따라서 다음과 같이 chain을 수정해 줘야 한다.

In [10]:
chain = {"context":retriever, "question": RunnablePassthrough()} | prompt | llm

chain.invoke("What is the name of Oldwyn’s mechanical doll?")

AIMessage(content="The name of Oldwyn's mechanical doll is Vesper.")

이렇게 "context":retriever 를 추가해주게 되면 retriever는 invoke에 입력해준 이 질문을 입력받아 호출(call)이 되고, 질문과 연관된 document들을 vector store에서 검색한 후 list로 뽑아서 prompt template의 context자리에 넣어줄 것이다.

또한 우리의 질문도 prompt template의 question property로 전달되게 해줘야 한다.<br>
이것은 RunnablePassthrough로 해결이 가능하다. RunnablePassthrough는 아주 간단한 기능의 class이다.<br>
입력값인 우리의 질문이 말그대로 통과하게(pass through) 해주는 역할을 한다.

이렇게 chain을 구성하면 에러가 없이 잘 작동된다.
